In [ ]:
# File: art_openings_brooklyn.py
from browser_use import Agent, ChatGoogle
from dotenv import load_dotenv
import os
import asyncio
import json
from datetime import datetime

# ===============================
# Setup
# ===============================

load_dotenv()
llm = ChatGoogle(model="gemini-2.5-flash")

# ===============================
# Gallery Sources
# ===============================

sources = [
    # 🖼 Bushwick
    "https://www.tigerstrikesasteroid.com/brooklyn",     # Tiger Strikes Asteroid
    "https://www.bushwickgallery.com",                   # Bushwick Gallery
    "https://www.brooklynartcave.com/events",            # Brooklyn Art Cave
    "https://carvalhopark.com/exhibitions",              # Carvalho Park
    "https://www.the-living-gallery.com/events",         # The Living Gallery
    "http://www.transmitter.nyc/",                      # Transmitter
    "https://www.activespacestudios.com/gallery",        # Active Space Studios Gallery

    # 🎨 Williamsburg
    "https://www.amant.org/exhibitions",                 # Amant
    "https://www.brooklyncc.com/dog-house-gallery",      # Dog House Gallery
    "https://awitanewyorkartmag.squarespace.com/announcements",  #awita
    "https://miriamgallery.com/",                       # miriam     
     # Awita New York Studio

    # 🌀 Ridgewood
    "https://www.tempestonweirfield.com/",               # Tempest (Ridgewood)
    "https://www.lorimoto.com/exhibitions",              # Lorimoto Gallery

    # 🌸 Maspeth
    "https://www.mrsgallery.com/exhibitions"             # Mrs. Gallery
]

visited_sources_file = "visited_galleries.json"
markdown_path = "brooklyn_openings_summary.md"
structured_path = "brooklyn_openings_data.json"

# ===============================
# Helpers
# ===============================

def load_visited_sources():
    if os.path.exists(visited_sources_file):
        with open(visited_sources_file, "r", encoding="utf-8") as f:
            return set(json.load(f))
    return set()

def save_visited_sources(visited):
    with open(visited_sources_file, "w", encoding="utf-8") as f:
        json.dump(list(visited), f, indent=2)

def get_remaining_sources():
    visited = load_visited_sources()
    return [s for s in sources if s not in visited]

def append_markdown(new_text, path=markdown_path):
    with open(path, "a", encoding="utf-8") as f:
        f.write("\n\n---\n\n")
        f.write(new_text)

def append_structured(new_data, path=structured_path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            existing = json.load(f)
    else:
        existing = []

    if isinstance(new_data, str):
        try:
            new_data = json.loads(new_data)
        except:
            new_data = [{"raw": new_data}]

    combined = existing + new_data
    with open(path, "w", encoding="utf-8") as f:
        json.dump(combined, f, indent=2)

# ===============================
# Agent Setup
# ===============================

def make_agent(remaining_sources):
    return Agent(
        task=(
            "Find the next exhibition opening or event for each listed art gallery "
            "in Bushwick, Williamsburg, Ridgewood, or Maspeth. "
            "Extract the event title, opening date, opening time, and if available, "
            "the exhibiting artist(s) or show theme. "
            "Ignore past exhibitions — only include future or current openings with 2025 dates. "
            "Return results as both a clean markdown summary and a structured JSON array "
            "with fields: gallery_name, address (if visible), event_name, opening_date, "
            "opening_time, artist, source_url. "
            f"Restrict browsing to these sites: {', '.join(remaining_sources)}."
        ),
        llm=llm,
        browser_config={
            "headless": True,
            "browser_type": "chromium",
            "browser_timeout": 90,
            "viewport_size": {"width": 1280, "height": 720},
            "extra_chromium_args": [
                "--no-sandbox",
                "--disable-dev-shm-usage",
                "--disable-gpu",
            ]
        }
    )

# ===============================
# Main Run
# ===============================

async def run_openings(batch_size=3):
    visited = load_visited_sources()
    remaining = get_remaining_sources()

    if not remaining:
        print("✅ All galleries already checked.")
        return

    next_batch = remaining[:batch_size]
    print(f"🎨 Checking galleries: {', '.join(next_batch)}")

    agent = make_agent(next_batch)

    try:
        history = await asyncio.wait_for(agent.run(max_steps=25), timeout=300)
    except asyncio.TimeoutError:
        print("❌ Agent timed out")
        return

    final_result = history.final_result()
    if final_result:
        append_markdown(f"## Gallery Batch ({', '.join(next_batch)})\n\n{final_result}")

        if hasattr(history, "structured_output") and history.structured_output:
            append_structured(history.structured_output)

        print(f"✅ Results from {next_batch} saved.")
    else:
        print("⚠️ No final result produced.")

    visited.update(next_batch)
    save_visited_sources(visited)

# ===============================
# Manual Execution
# ===============================

if __name__ == "__main__":
    asyncio.run(run_openings())
